<a href="https://colab.research.google.com/github/housemLassoued/ML-deployment/blob/main/Recyclable_and_Household_Waste_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"housemlassoued","key":"6a8af7b733d882893bd4dfe554037bef"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d alistairking/recyclable-and-household-waste-classification

Dataset URL: https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification
License(s): MIT
 99% 909M/920M [00:04<00:00, 247MB/s]
100% 920M/920M [00:04<00:00, 222MB/s]


In [ ]:
! unzip /content/recyclable-and-household-waste-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/images/plastic_shopping_bags/default/Image_1.png  
  inflating: images/images/plastic_shopping_bags/default/Image_10.png  
  inflating: images/images/plastic_shopping_bags/default/Image_100.png  
  inflating: images/images/plastic_shopping_bags/default/Image_101.png  
  inflating: images/images/plastic_shopping_bags/default/Image_102.png  
  inflating: images/images/plastic_shopping_bags/default/Image_103.png  
  inflating: images/images/plastic_shopping_bags/default/Image_104.png  
  inflating: images/images/plastic_shopping_bags/default/Image_105.png  
  inflating: images/images/plastic_shopping_bags/default/Image_106.png  
  inflating: images/images/plastic_shopping_bags/default/Image_107.png  
  inflating: images/images/plastic_shopping_bags/default/Image_108.png  
  inflating: images/images/plastic_shopping_bags/default/Image_109.png  
  inflating: images/images/plastic_shopping_bags/default/Image_11.png  
  in

In [ ]:
import os
import shutil
import glob
import random

# Paths
base_path = '/content/images/images'
output_base = 'dataset_split'

# Define waste categories and their type
plastic_items = [
    'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers',
    'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws',
    'plastic_trash_bags', 'plastic_water_bottles', 'disposable_plastic_cutlery'
]

not_plastic_items = [
    'aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes',
    'cardboard_packaging', 'clothing', 'coffee_grounds', 'eggshells', 'food_waste',
    'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars',
    'magazines', 'newspaper', 'office_paper', 'paper_cups', 'shoes', 'steel_food_cans',
    'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags'
]

# Ensure output folders are created
for split in ['train', 'validation', 'test']:
    os.makedirs(os.path.join(output_base, split, 'plastic'), exist_ok=True)
    os.makedirs(os.path.join(output_base, split, 'not_plastic'), exist_ok=True)

# Define split ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

def organize_data(items, category_name):
    for item in items:
        for condition in ['default', 'real_world']:
            item_path = os.path.join(base_path, item, condition, '*.png')
            images = glob.glob(item_path)
            random.shuffle(images)

            # Calculate sizes for each split
            train_size = int(len(images) * train_ratio)
            validation_size = int(len(images) * validation_ratio)

            # Create splits
            split_sets = {
                'train': images[:train_size],
                'validation': images[train_size:train_size + validation_size],
                'test': images[train_size + validation_size:]
            }

            # Copy images to their respective split folders
            for split, img_paths in split_sets.items():
                dest_dir = os.path.join(output_base, split, category_name, item)
                os.makedirs(dest_dir, exist_ok=True)
                for img_path in img_paths:
                    shutil.copy(img_path, dest_dir)

# Organize the dataset into `plastic` and `not_plastic`
organize_data(plastic_items, 'plastic')
organize_data(not_plastic_items, 'not_plastic')

print("Dataset organized and split into train, validation, and test folders.")


Dataset organized and split into train, validation, and test folders.


In [ ]:
import os
import glob

# Define the lists to store images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []
validation_images = []
validation_labels = []

# Define paths
base_path = '/content/dataset_split'

# Assign unique labels to plastic items and a single label for all not_plastic items
plastic_labels = {item: idx for idx, item in enumerate(plastic_items)}
not_plastic_label = len(plastic_items)  # One common label for all "not plastic" items

# Function to append images and labels
def append_images_labels(images, label, img_list, label_list):
    img_list.extend(images)
    label_list.extend([label] * len(images))

# Process each part (train, validation, test)
for part in ['train', 'validation', 'test']:
    # Determine the appropriate list for images and labels
    if part == 'train':
        img_list = train_images
        label_list = train_labels
    elif part == 'validation':
        img_list = validation_images
        label_list = validation_labels
    else:
        img_list = test_images
        label_list = test_labels

    # Process plastic items with unique labels
    for item in plastic_items:
        path = os.path.join(base_path, part, 'plastic', item, '*')
        images = glob.glob(path)
        append_images_labels(images, plastic_labels[item], img_list, label_list)

    # Process not plastic items with a common label
    for item in not_plastic_items:
        path = os.path.join(base_path, part, 'not_plastic', item, '*')
        images = glob.glob(path)
        append_images_labels(images, not_plastic_label, img_list, label_list)

# Example output: Check the number of images and labels in each dataset part
print(f"Train set: {len(train_images)} images, {len(train_labels)} labels")
print(f"Validation set: {len(validation_images)} images, {len(validation_labels)} labels")
print(f"Test set: {len(test_images)} images, {len(test_labels)} labels")

Train set: 6822 images, 6822 labels
Validation set: 2063 images, 2063 labels
Test set: 2100 images, 2100 labels


In [ ]:
import cv2

# Function to resize and normalize images
def process_images(image_paths):
    processed_images = []
    for image_path in image_paths:
        image = cv2.imread(image_path)  # Load the image
        if image is not None:
            resized_image = cv2.resize(image, (224, 224))  # Resize to 224x224
            normalized_image = resized_image / 255.0  # Normalize to [0, 1]
            processed_images.append(normalized_image)
    return processed_images

# Apply processing to each dataset
train_images_processed = process_images(train_images)
test_images_processed = process_images(test_images)
validation_images_processed = process_images(validation_images)

# Example: Check the size and range of a processed image
print(f"Processed train image shape: {train_images_processed[0].shape}")
print(f"Pixel value range: min={train_images_processed[0].min()}, max={train_images_processed[0].max()}")

Processed train image shape: (224, 224, 3)
Pixel value range: min=0.0, max=1.0


In [ ]:
import numpy as np

# Convert lists of images and labels into numpy arrays
def to_numpy(images, labels):
    # Stack images into a single numpy array
    images_np = np.array(images, dtype=np.float32)
    # Convert labels to numpy array
    labels_np = np.array(labels, dtype=np.int32)
    return images_np, labels_np

# Convert train, test, and validation sets
train_images_np, train_labels_np = to_numpy(train_images_processed, train_labels)
test_images_np, test_labels_np = to_numpy(test_images_processed, test_labels)
validation_images_np, validation_labels_np = to_numpy(validation_images_processed, validation_labels)

# Check the shapes of the numpy arrays
print("Train images shape:", train_images_np.shape)
print("Train labels shape:", train_labels_np.shape)
print("Test images shape:", test_images_np.shape)
print("Test labels shape:", test_labels_np.shape)
print("Validation images shape:", validation_images_np.shape)
print("Validation labels shape:", validation_labels_np.shape)

Train images shape: (6822, 224, 224, 3)
Train labels shape: (6822,)
Test images shape: (2100, 224, 224, 3)
Test labels shape: (2100,)
Validation images shape: (2063, 224, 224, 3)
Validation labels shape: (2063,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load VGG16 with pretrained weights, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of VGG16 to use it as a feature extractor
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of VGG16 for our classification task
x = base_model.output
x = Flatten()(x)  # Flatten the output of the VGG16 base
x = Dense(256, activation='relu')(x)  # Fully connected layer
x = Dense(128, activation='relu')(x)  # Another fully connected layer
# Output layer with 10 classes (plastic classes and not plastic)
output = Dense(10, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with your dataset
model.fit(train_images_np, train_labels_np, epochs=10, batch_size=32,
          validation_data=(validation_images_np, validation_labels_np))


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
214/214 [==============================] - 168s 782ms/step - loss: 0.7661 - accuracy: 0.7875 - val_loss: 0.4678 - val_accuracy: 0.8667
Epoch 2/10
214/214 [==============================] - 167s 782ms/step - loss: 0.3380 - accuracy: 0.8996 - val_loss: 0.3963 - val_accuracy: 0.8837
Epoch 3/10
214/214 [==============================] - 169s 791ms/step - loss: 0.2046 - accuracy: 0.9447 - val_loss: 0.3651 - val_accuracy: 0.8880
Epoch 4/10
214/214 [==============================] - 169s 793ms/step - loss: 0.1305 - accuracy: 0.9648 - val_loss: 0.3454 - val_accuracy: 0.9016
Epoch 5/10
214/214 [==============================] - 170s 795ms/step - loss: 0.0923 - accuracy: 0.9764 - val_loss: 0.3568 - val_accuracy: 0.9069
Epoch 6/10
214/214 [==============================] - 169s 791ms/step - loss: 0.0622 - accuracy: 0.9848 - val_loss: 0.3622 - val_accuracy: 0.9084
Epoch 7/10
214/214 [==============================] - 173s 

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images_np, test_labels_np, batch_size=32)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on the test set
predictions = model.predict(test_images_np)
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the highest probability

# Print out the first 10 predictions vs actual labels
for i in range(10):
    print(f"Predicted: {predicted_labels[i]}, Actual: {test_labels_np[i]}")


66/66 [==============================] - 40s 599ms/step - loss: 0.3651 - accuracy: 0.9105
Test Loss: 0.36509037017822266
Test Accuracy: 0.9104762077331543
66/66 [==============================] - 41s 612ms/step
Predicted: 9, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0
Predicted: 0, Actual: 0


In [ ]:
from sklearn.metrics import classification_report

# Predict the labels for the test dataset
predictions = model.predict(test_images_np)
predicted_labels = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Generate and display the classification report
class_names = ['plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers',
    'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws',
    'plastic_trash_bags', 'plastic_water_bottles', 'disposable_plastic_cutlery', 'Not Plastic']
report = classification_report(test_labels_np, predicted_labels, target_names=class_names)
print(report)


66/66 [==============================] - 39s 586ms/step
                            precision    recall  f1-score   support

          plastic_cup_lids       0.81      0.82      0.82        73
 plastic_detergent_bottles       0.93      0.77      0.85        71
   plastic_food_containers       0.91      0.74      0.82        69
     plastic_shopping_bags       0.90      0.76      0.82        70
      plastic_soda_bottles       0.79      0.80      0.79        70
            plastic_straws       0.87      0.78      0.82        67
        plastic_trash_bags       0.96      0.71      0.82        69
     plastic_water_bottles       0.77      0.66      0.71        67
disposable_plastic_cutlery       0.92      0.79      0.85        73
               Not Plastic       0.93      0.97      0.95      1471

                  accuracy                           0.91      2100
                 macro avg       0.88      0.78      0.82      2100
              weighted avg       0.91      0.91      0.91 